# Análise das Intents

Este jupyter-notebook vai auxiliar na análise de detecção de intenções de seu chatbot.

### Configurando e Imports

In [1]:
from IPython.display import IFrame

import rasa_nlu
print("rasa_nlu: {}".format(rasa_nlu.__version__))

rasa_nlu: 0.15.0


### Treinando o modelo do Rasa NLU

* Para avaliar o bot o primeiro passo é treiner o seu chatbot. Mas não é necessário treinar a parte de conversão completa (rasa_core) apenas a parte de interpretação de mensagens (rasa_nlu).

* O comando `train-nlu` do Makefile executa o treinamento apenas do `rasa_nlu`

In [2]:
!make train-nlu -C $COACH_DIR_PATH

make: Entering directory '/work/coach'
python3 -m rasa_nlu.train -c nlu_config.yml --fixed_model_name current \
--data data/intents/ -o /src_models --project nlu --verbose
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.train' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-07-21 15:59:31 INFO     rasa_nlu.training_data.loading  - Training data format of data/intents/actions.md is md
2019-07-21 15:59:31 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 6 (1 distinct intents)
	- Found intents: 'action_test'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

2019-07-21 15:59:31 INFO     rasa_nlu.training_data.loading  - Training data format of data/intents/aleatorio.md is md
2019-07-21 15:59:31 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 197 (2

### Métodos de avaliação do chatbot

* O Rasa fornece vários métodos de avaliação e validação das `intents`, para verificar como utiliza-los, cada método fornece um log, imagem, gráfico ou arquivo com dados relevantes para interpretação do chatbot.

* O comando `!python -m rasa_nlu.test` é a base para a avaliação do chatbot.

* Na célula abaixo a flag `-h` foi utilizada para mostrar as funções e a forma de uso de cada uma delas, mude seus valores e flags para ter as informações desejadas na sua análise.

* Atualmente o Rasa possui 2 modos, `evaluation` e `crossvalidation` que tem seções decicadas a eles neste jupyter notebook.

In [3]:
!python -m rasa_nlu.test -h

/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: test.py [-h] [--debug] [-v] -d DATA [--mode MODE] [-c CONFIG]
               [-m MODEL] [-f FOLDS] [--report [REPORT]]
               [--successes [SUCCESSES]] [--errors ERRORS]
               [--histogram HISTOGRAM] [--confmat CONFMAT]

evaluate a Rasa NLU pipeline with cross validation or on external data

optional arguments:
  -h, --help            show this help message and exit
  --debug               Print lots of debugging statements. Sets logging level
                        to DEBUG
  -v, --verbose         Be verbose. Sets logging level to INFO
  -d DATA, --data DATA  file containing training/evaluation data
  --mode MODE           evaluation|crossvalidation (evaluate pretrained model
                        or train model by cross

### Evaluation

* O comando abaixo gera informações relevates para a validação das `intents` são elas:
    * Matriz de confusão
    * Histograma
    * Erros de intents

In [4]:
!rm erros.json
!python -m rasa_nlu.test -d $COACH_INTENTS_PATH -m $COACH_MODELS_NLU_PATH --mode evaluation

rm: cannot remove 'erros.json': No such file or directory
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-07-21 15:59:46.381460: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-07-21 15:59:46.398893: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2905000000 Hz
2019-07-21 15:59:46.399579: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x557727013d80 executing computations on platform Host. Devices:
2019-07-21 15:59:46.399669: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-07-21 15:59:46 WARNING  tensorflow  - From /usr/local/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1

#### Matriz de Confusão
* A matriz de confusão mostra a correlação entre as intents.
* A diagonal principal tem forte correlação pois mostra a relação de uma intent **com ela mesma**
* O ideal é que não haja **nenhum valor** diferente de **0 fora da diagonal principal**.

`Execute este notebook no Firefox para visualizar o PDF da matriz de confusão`

In [5]:
IFrame(src='./confmat.png', width=900, height=700)

#### Erros
* O arquivo `erros,json` mostra os erros encontrados. Este arquivo lista os mesmos erros mostrados na **matriz de confusão**, então caso ela fique muito grande, não se preocupe, pois você pode apenas procurar os erros no arquivo gerado.
* Os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.
* Caso o arquivo não seja gerado significa que não foram encontrados erros.

In [6]:
%cat errors.json

[
    {
        "text": "e o c#",
        "intent": "linguagens",
        "intent_prediction": {
            "name": "",
            "confidence": 0.0
        }
    },
    {
        "text": "tudo bom",
        "intent": "cumprimentar",
        "intent_prediction": {
            "name": "tudo_bem",
            "confidence": 0.862882137298584
        }
    },
    {
        "text": "como vai",
        "intent": "tudo_bem",
        "intent_prediction": {
            "name": "cumprimentar",
            "confidence": 0.8922472596168518
        }
    }
]

#### Histograma

* O histograma contém a distribuição da predições das `intents`

In [7]:
IFrame(src='./hist.png', width=900, height=700)

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `intents` são:

* [Evaluation](https://rasa.com/docs/nlu/evaluation/)